In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
credit.head(1)

In [ ]:
movies=movies.merge(credit, on='title')

In [ ]:
movies.head(1)

In [ ]:
# genres
# id
# keywords
# overview
# title
# overview
# cast
# crew
movies=movies[['id','genres','title' ,'keywords','overview','cast','crew']]

In [ ]:
pd.set_option('display.max_columns',234)

In [ ]:
movies

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.head(1)

In [ ]:
movies

In [ ]:
movies.iloc[0].genres

In [ ]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
        

In [ ]:
import ast

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies

In [ ]:
def convert2(obj): #for practice
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
movies['keywords']=movies['keywords'].apply(convert2)

In [ ]:
movies

In [ ]:
movies.cast

In [ ]:
def convert3(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            l.append(i['character'])
            counter+=1
        else:
            break
    return l

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies

In [ ]:
movies['crew'][0]

In [ ]:
def conver4(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']== 'Director':
            
            l.append(i['name'])
            break
    return l
        

In [ ]:
movies['crew']=movies['crew'].apply(conver4)

In [ ]:
movies

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies

In [ ]:
movies['tags']= movies['overview']+ movies['genres']+ movies['keywords']+movies['cast']+ movies['crew']

In [ ]:
new_df=movies[['id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors= cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

# stemming

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
        

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
new_df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        
    

In [ ]:
recommend("Aliens")

In [ ]:
similarity[0].shape

In [ ]:
import numpy as np
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('sim.pkl','wb'))